In [4]:
"""
Initial setup: Imported core libraries for data processing and model training
 - Imported pandas for data loading and preprocessing
 - Imported LinearRegression from sklearn.linear_model for model training
 - Imported mean_squared_error from sklearn.metrics for model evaluation (RMSE)
 - Imported numpy for numerical operations (e.g., handling arrays, checking for NaNs)
"""
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [5]:
# Load January 2023 Yellow Taxi dataset
df1 = pd.read_parquet('../data/yellow_tripdata_2023-01.parquet')

In [6]:
df1.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [7]:
# Normalize column names to lowercase. In order to be able to merge monthly datasets without conflicts
df1.columns = df1.columns.str.lower()

In [8]:
# Load February 2023 Yellow Taxi dataset
df2 = pd.read_parquet('../data/yellow_tripdata_2023-02.parquet')

In [9]:
df2.columns = df2.columns.str.lower()

In [10]:
df2.head()

,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


In [11]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   vendorid               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   ratecodeid             float64       
 6   store_and_fwd_flag     object        
 7   pulocationid           int64         
 8   dolocationid           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [12]:
# Adding a new column called "duration", based on drop-off and pickup timestamps
df1["duration"] = (df1["tpep_dropoff_datetime"] - df1["tpep_pickup_datetime"]).dt.total_seconds()/60

In [13]:
df1[["duration"]].head(10)

,duration
0,8.433333
1,6.316667
2,12.750000
3,9.616667
4,10.833333
5,12.300000
6,10.450000
7,22.733333
8,14.933333
9,10.900000


In [14]:
# Calculating the standard deviation of the trips duration in January using the "duration" column
df1["duration"].std()

np.float64(42.59435124195458)

In [15]:
# Dropping outliers. Checking the distribution of the duration variable
duration_outliers_share = df1[(df1["duration"] >= 1) & (df1["duration"] <= 60)].shape[0]/df1["duration"].count()
print("Fraction of the records will be left after dropping outliers:", duration_outliers_share)

Fraction of the records will be left after dropping outliers: 0.9812202822125979


In [16]:
# Dropping operation, to remove all defined outliers
df1_cleaned = df1[(df1["duration"] >= 1) & (df1["duration"] <= 60)]

In [17]:
# Applying one-hot encoding to the pickup and drop-off location IDs. Only 2 features will be using for our model. Turn the dataframe into a list of dictionaries (re-casting the ids to strings - otherwise it will label encode them)
df_small = df1_cleaned[["pulocationid", "dolocationid"]].copy()
df_small["pulocationid"] = df_small["pulocationid"].astype(str)
df_small["dolocationid"] = df_small["dolocationid"].astype(str)
dicts = df_small.to_dict(orient="records")
dicts[0]

{'pulocationid': '161', 'dolocationid': '141'}

In [18]:
# Fit a dictionary vectorizer. Get a feature matrix from it
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer()
X = dv.fit_transform(dicts)

In [19]:
# Display the dimensionality of this matrix (number of columns)
X.shape

(3009173, 515)

In [20]:
# Extract target variable for modeling
y = df1_cleaned['duration'].values

In [21]:
# Train the model
model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [22]:
print(type(y), y.shape)

<class 'numpy.ndarray'> (3009173,)


In [23]:
# Predict the target metric and evaluate
y_pred = model.predict(X)

In [24]:
print(type(y_pred), y_pred.shape)

<class 'numpy.ndarray'> (3009173,)


In [25]:
print(np.isnan(y).sum(), np.isnan(y_pred).sum())

0 0


In [26]:
# Calculating the RMSE metric
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mse)
print(f"RMSE on training data: {rmse:.2f}")

RMSE on training data: 7.65


In [27]:
# Replicating all the steps for February 2023
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2913955 entries, 0 to 2913954
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   vendorid               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   ratecodeid             float64       
 6   store_and_fwd_flag     object        
 7   pulocationid           int32         
 8   dolocationid           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [28]:
df2["duration"] = (df2["tpep_dropoff_datetime"] - df2["tpep_pickup_datetime"]).dt.total_seconds()/60

In [29]:
duration_outliers_february_2003_share = df2[(df2["duration"] >= 1) & (df2["duration"] <= 60)].shape[0]/df2["duration"].count()
print("Fraction of the records will be left after dropping outliers:", duration_outliers_february_2003_share)

Fraction of the records will be left after dropping outliers: 0.9800944077722545


In [30]:
df2_cleaned = df2[(df2["duration"] >= 1) & (df2["duration"] <= 60)]

In [31]:
df2_small = df2_cleaned[["pulocationid", "dolocationid"]].copy()
df2_small["pulocationid"] = df2_small["pulocationid"].astype(str)
df2_small["dolocationid"] = df2_small["dolocationid"].astype(str)

In [32]:
dicts_df2 = df2_small.to_dict(orient="records")

In [33]:
dv2 = DictVectorizer()
X2 = dv2.fit_transform(dicts_df2)

In [34]:
y_february = df2_cleaned['duration'].values

In [35]:
model_february = LinearRegression()
model.fit(X2, y_february)

LinearRegression()

In [36]:
y_february_pred = model.predict(X2)

In [37]:
mse_february = mean_squared_error(y_february, y_february_pred)
rmse_february = np.sqrt(mse_february)
print(f"RMSE on training data: {rmse_february:.2f}")

RMSE on training data: 7.78
